# **Regression Evaluate_Predict Examples**

In [ ]:
print("ML.EVALUATE results for bqml_arrdelay_model:")
display(evaluation_results)

ML.EVALUATE results for bqml_arrdelay_model:


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,9.17431,568.936212,1.41345,6.467743,0.497171,0.497209


In [ ]:
import pandas as pd

new_hypothetical_flights = [
    {
        'DepDelay': 15,
        'Distance': 750,
        'Reporting_Airline': 'UA',
        'Origin': 'DEN',
        'Dest': 'LAX',
        'DayOfWeek': 2   # Tuesday
    },
    {
        'DepDelay': 0,
        'Distance': 200,
        'Reporting_Airline': 'WN',
        'Origin': 'DAL',
        'Dest': 'HOU',
        'DayOfWeek': 4   # Thursday
    },
    {
        'DepDelay': 60,
        'Distance': 1500,
        'Reporting_Airline': 'AA',
        'Origin': 'DFW',
        'Dest': 'SFO',
        'DayOfWeek': 6   # Saturday
    }
]

new_hypothetical_flights_df = pd.DataFrame(new_hypothetical_flights)
print("New Hypothetical Flight Scenarios:")
display(new_hypothetical_flights_df)

New Hypothetical Flight Scenarios:


,DepDelay,Distance,Reporting_Airline,Origin,Dest,DayOfWeek
0,15,750,UA,DEN,LAX,2
1,0,200,WN,DAL,HOU,4
2,60,1500,AA,DFW,SFO,6


In [ ]:
temp_new_table_id = f"{PROJECT_ID}.flights.temp_new_hypothetical_flights"
job = bq.load_table_from_dataframe(new_hypothetical_flights_df, temp_new_table_id)
job.result()  # Wait for the job to complete
print(f"DataFrame uploaded to BigQuery table: {temp_new_table_id}")

DataFrame uploaded to BigQuery table: mgmt-467-project-1.flights.temp_new_hypothetical_flights


In [ ]:
predict_query = f"""
SELECT *
FROM ML.PREDICT(MODEL `{PROJECT_ID}.flights.{model_name}`,
  (SELECT * FROM `{temp_new_table_id}`))
"""

new_prediction_results_df = bq.query(predict_query).result().to_dataframe()
print("Predictions for new hypothetical flights generated:")
display(new_prediction_results_df)

Predictions for new hypothetical flights generated:


,predicted_ArrDelay,DepDelay,Distance,Reporting_Airline,Origin,Dest,DayOfWeek
0,13.309999,15,750,UA,DEN,LAX,2
1,-2.135674,0,200,WN,DAL,HOU,4
2,55.994159,60,1500,AA,DFW,SFO,6


# **Classification Evaluate_predict Examples**

In [ ]:
print("ML.EVALUATE results for bqml_diverted_model_weighted:")
display(evaluation_results_diverted_weighted)

ML.EVALUATE results for bqml_diverted_model_weighted:


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.004432,0.947368,0.59869,0.008824,0.693004,0.837579


In [ ]:
import pandas as pd

new_hypothetical_flights_classification = [
    {
        'DepDelay': 10,
        'Distance': 300,
        'Reporting_Airline': 'WN',
        'Origin': 'MDW',
        'Dest': 'STL',
        'DayOfWeek': 3   # Wednesday
    },
    {
        'DepDelay': 120,
        'Distance': 2000,
        'Reporting_Airline': 'UA',
        'Origin': 'EWR',
        'Dest': 'SFO',
        'DayOfWeek': 5   # Friday
    },
    {
        'DepDelay': 0,
        'Distance': 150,
        'Reporting_Airline': 'AA',
        'Origin': 'DFW',
        'Dest': 'AUS',
        'DayOfWeek': 1   # Monday
    }
]

new_hypothetical_flights_classification_df = pd.DataFrame(new_hypothetical_flights_classification)
print("New Hypothetical Flight Scenarios for Classification:")
display(new_hypothetical_flights_classification_df)

New Hypothetical Flight Scenarios for Classification:


,DepDelay,Distance,Reporting_Airline,Origin,Dest,DayOfWeek
0,10,300,WN,MDW,STL,3
1,120,2000,UA,EWR,SFO,5
2,0,150,AA,DFW,AUS,1


In [ ]:
temp_new_table_id_classification = f"{PROJECT_ID}.flights.temp_new_hypothetical_flights_classification"
job = bq.load_table_from_dataframe(new_hypothetical_flights_classification_df, temp_new_table_id_classification)
job.result()  # Wait for the job to complete
print(f"DataFrame uploaded to BigQuery table: {temp_new_table_id_classification}")

DataFrame uploaded to BigQuery table: mgmt-467-project-1.flights.temp_new_hypothetical_flights_classification


In [ ]:
predict_classification_query = f"""
SELECT
    predicted_Diverted,
    (SELECT prob FROM UNNEST(predicted_Diverted_probs) WHERE label = 1) AS probability_of_diverted
FROM
    ML.PREDICT(MODEL `{PROJECT_ID}.flights.{model_name_diverted_weighted}`, (SELECT * FROM `{temp_new_table_id_classification}`))
"""

new_classification_prediction_results_df = bq.query(predict_classification_query).result().to_dataframe()
print("Predictions for new hypothetical flights (classification) generated:")
display(new_classification_prediction_results_df.head())

Predictions for new hypothetical flights (classification) generated:


,predicted_Diverted,probability_of_diverted
0,0,0.498394
1,1,0.515890
2,0,0.497740


In [ ]:
print("\n--- Interpretation of Classification Predictions ---\n")

for index, row in new_classification_prediction_results_df.iterrows():
    predicted_diverted = row['predicted_Diverted']
    probability_diverted = row['probability_of_diverted']

    original_input = new_hypothetical_flights_classification_df.loc[index]

    print(f"--- Hypothetical Flight Scenario {index + 1} ---")
    print(f"Original Input:")
    print(f"  DepDelay: {original_input['DepDelay']} minutes")
    print(f"  Distance: {original_input['Distance']} miles")
    print(f"  Reporting_Airline: {original_input['Reporting_Airline']}")
    print(f"  Origin: {original_input['Origin']}")
    print(f"  Dest: {original_input['Dest']}")
    print(f"  DayOfWeek: {original_input['DayOfWeek']}")

    print(f"Predicted Diverted Status: {predicted_diverted} (1 = Diverted, 0 = Not Diverted)")
    print(f"Probability of Diverted (label=1): {probability_diverted:.4f}\n")

    print("Interpretation:")
    if predicted_diverted == 1:
        print(f"  The model predicts this flight is likely to be diverted (predicted as 1) with a high probability of {probability_diverted:.4f}.")
        print("  This could be due to significant departure delays, long distance, or a combination of factors making diversion more probable for this route/airline.")
    else:
        print(f"  The model predicts this flight is not likely to be diverted (predicted as 0) with a probability of {probability_diverted:.4f} for diversion.")
        print("  Factors like shorter delays, shorter distances, or specific airline/route characteristics might contribute to a lower likelihood of diversion.")
    print("\n")


--- Interpretation of Classification Predictions ---

--- Hypothetical Flight Scenario 1 ---
Original Input:
  DepDelay: 10 minutes
  Distance: 300 miles
  Reporting_Airline: WN
  Origin: MDW
  Dest: STL
  DayOfWeek: 3
Predicted Diverted Status: 0 (1 = Diverted, 0 = Not Diverted)
Probability of Diverted (label=1): 0.4984

Interpretation:
  The model predicts this flight is not likely to be diverted (predicted as 0) with a probability of 0.4984 for diversion.
  Factors like shorter delays, shorter distances, or specific airline/route characteristics might contribute to a lower likelihood of diversion.


--- Hypothetical Flight Scenario 2 ---
Original Input:
  DepDelay: 120 minutes
  Distance: 2000 miles
  Reporting_Airline: UA
  Origin: EWR
  Dest: SFO
  DayOfWeek: 5
Predicted Diverted Status: 1 (1 = Diverted, 0 = Not Diverted)
Probability of Diverted (label=1): 0.5159

Interpretation:
  The model predicts this flight is likely to be diverted (predicted as 1) with a high probability of